In [7]:
import requests
import pandas as pd

tokenCanvas="10747~lGBNJqxBXPlYOYvb8lHvCNn9yXQo91Gu6LC3vfySqhrVl0F2hjzKNyD8KBVOsGKL"
parametreCanvas = {'per_page': '100'}
hodeCanvas = {'Authorization': 'Bearer ' + tokenCanvas}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

gruppe_url = f"https://hvl.instructure.com:443/api/v1/courses/10747/groups"
resultat = requests.get(gruppe_url, )

dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(gruppe_url, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            if element['members_count'] > 0:
                dataliste.append([element['id'], element['name'], element['members_count']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'antal'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

In [8]:
alledata

,id,name,antal
0,127438,Isolert gruppe,1
1,80951,Praksis 1,2
2,80950,Praksis 2,1
3,101388,Aasmunds testgruppe,4
4,127435,Gruppe A,2
5,127433,Gruppe B,2
